In [1]:
import spuco
import torch # use kernel python 3.10.13 for my machine

from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from spuco.models import model_factory
from spuco.robust_train import ERM, GroupBalanceBatchERM
from spuco.group_inference import ClusterAlg
from spuco.utils import set_seed
from spuco.evaluate import Evaluator
from spuco.group_inference import Cluster, ClusterAlg 

from torch.optim import SGD
import torchvision.transforms as T

import os

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [5]:
set_seed(0)

In [6]:
root_dir = os.getcwd()
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root=root_dir,
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

100%|██████████| 48004/48004 [00:17<00:00, 2717.97it/s]


In [17]:
valset = SpuCoMNIST(
    root=root_dir,
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val",
)
valset.initialize()

100%|██████████| 11996/11996 [00:06<00:00, 1972.38it/s]


In [8]:
testset = SpuCoMNIST(
    root=root_dir,
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:03<00:00, 2695.29it/s]


In [9]:
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)

In [10]:
erm = ERM(
    model=model, 
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True),
    num_epochs=1,
    device = device,
    verbose=True,
)

In [47]:
T.ToPILImage()(trainset[1000][0]).resize((28,28)) # taken from spuco_mnist_erm.ipynb in the github

In [40]:
print(len(valset))
print(len(trainset))
print(len(testset))

11996
48004
10000


In [15]:
erm.train() # gives error: Can't pickle local object 'SpuCoMNIST.load_data.<locals>.<lambda>'; can't figure out why or how to fix

Epoch 0:   0%|          | 0/751 [00:00<?, ?batch/s]


AttributeError: Can't pickle local object 'SpuCoMNIST.load_data.<locals>.<lambda>'

In [16]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

# gives same error: Can't pickle local object 'SpuCoMNIST.load_data.<locals>.<lambda>'; can't figure out why or how to fix

Evaluating group-wise accuracy:   0%|          | 0/25 [00:00<?, ?it/s]


AttributeError: Can't pickle local object 'SpuCoMNIST.load_data.<locals>.<lambda>'